In [4]:
# import sys
# !{sys.executable} -m pip install autogluon.timeseries

  Using cached gluonts-0.15.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached boto3-1.35.56-py3-none-any.whl.metadata (6.7 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached window_ops-0.0.15-py3-none-any.whl.metadata (6.8 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached botocore-1.35.56-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached py4j-0.10.9

In [11]:
import torch
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


In [12]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np

# Read data

In [13]:
my_data = pd.read_csv('../GD030A_S.csv')

In [14]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [15]:
traffic_full = recover_timestamp(my_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


In [35]:
train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
test_set = traffic_full['2023-07-01 00:00:00':]
# train_set = traffic_full[:'2022-02-28 23:00:00']
# valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
# test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

Proportion of train_set : 0.2067
Proportion of valid_set : 0.0623
Proportion of test_set : 0.0630


In [36]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates sequences for time series data, excluding any sequences containing NaN values.

    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.

    Returns:
    - sequences_df: pandas DataFrame containing all sequences with input lags and output leads,
                    and an 'item_id' column to separate each sequence, including timestamps.
    """
    sequences = []
    item_id = 0  # To assign unique ID to each sequence

    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow'].values
        X_timestamps = data.iloc[i - input_length:i].index
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow'].values
        y_timestamps = data.iloc[i:i + forecast_horizon].index

        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            # Convert X_seq and y_seq to DataFrames, including their timestamps
            x_df = pd.DataFrame({'target': X_seq, 'timestamp': X_timestamps})
            y_df = pd.DataFrame({'target': y_seq, 'timestamp': y_timestamps})

            # Concatenate X_df and y_df
            seq_df = pd.concat([x_df, y_df], ignore_index=True)

            # Add 'item_id' column
            seq_df['item_id'] = str(item_id)
            item_id += 1  # Increment item_id

            # Append to list
            sequences.append(seq_df)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs

    # Concatenate all sequences into one DataFrame
    sequences_df = pd.concat(sequences)
    sequences_df['item_id'] = sequences_df['item_id'].astype('str')
    sequences_df['target'] = sequences_df['target'].astype('float32')

    return sequences_df

In [37]:
input_lengths = [24 * i for i in range(1, 22)]
prediction_length = 6

In [38]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")

    # Create sequences with forecast_horizon=6
    test = create_sequences(test_set, length, forecast_horizon=6)

    # Store in the dictionary
    data_dict[length]['test'] = test

    # Print shapes and ensure no NaNs
    print(f"  test shape: {test.index.max()+1}, {len(test.item_id.unique())}\n")

Processing input length: 24
  test shape: 30, 1719

Processing input length: 48
  test shape: 54, 1604

Processing input length: 72
  test shape: 78, 1508

Processing input length: 96
  test shape: 102, 1412

Processing input length: 120
  test shape: 126, 1316

Processing input length: 144
  test shape: 150, 1220

Processing input length: 168
  test shape: 174, 1124

Processing input length: 192
  test shape: 198, 1028

Processing input length: 216
  test shape: 222, 932

Processing input length: 240
  test shape: 246, 836

Processing input length: 264
  test shape: 270, 740

Processing input length: 288
  test shape: 294, 660

Processing input length: 312
  test shape: 318, 588

Processing input length: 336
  test shape: 342, 516

Processing input length: 360
  test shape: 366, 444

Processing input length: 384
  test shape: 390, 390

Processing input length: 408
  test shape: 414, 342

Processing input length: 432
  test shape: 438, 311

Processing input length: 456
  test shape: 46

# Zero-shot

In [39]:
import time

In [40]:
def chronos_zero_shot(input_lengths, prediction_length, model_type):
    results_dict = defaultdict(dict)
    inference_results = []

    for length in input_lengths:
        logging.info(f"Processing input length: {length}")

        # prepare the data
        test_data = TimeSeriesDataFrame(data_dict[length]['test'])
        test_data_ready, test_data = test_data.train_test_split(prediction_length)

        predictor = TimeSeriesPredictor(prediction_length=prediction_length,verbosity=2).fit(test_data_ready, presets=model_type)
        
        # Record start time
        start_time = time.time()
        
        predictions = predictor.predict(test_data_ready)
        
        # Record end time and compute the inference time
        inference_time = time.time() - start_time
        logging.info(f"\nInference time: {inference_time:.6f} seconds")
        
        ground_truth_df = test_data.groupby(level='item_id').tail(6)
        predicted_df = predictions[['mean']]

        results_dict[length]['Predicted_Results'] = predicted_df
        results_dict[length]['Gound_Truth'] = ground_truth_df

        inference_results.append({
            "length": length,
            "inference_time": inference_time
        })

        torch.cuda.empty_cache()

    # Convert the results into a DataFrame
    df_inference = pd.DataFrame(inference_results)
    logging.info(f"{model_type}")
    logging.info("\n" + df_inference.to_string(index=False))

    return results_dict

In [44]:
torch.cuda.empty_cache()

In [41]:
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(asctime)s - %(message)s',  # Customize the log message format
    handlers=[
        logging.FileHandler('chronos.log'),  # Log messages to 'output.log'
        logging.StreamHandler()             # Also output to console/notebook
    ]
)

In [ ]:
# entire data
results_dict1 = chronos_zero_shot(input_lengths, prediction_length, "chronos_tiny")
results_dict2 = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")
results_dict3 = chronos_zero_shot(input_lengths, prediction_length, "chronos_small")
results_dict4 = chronos_zero_shot(input_lengths, prediction_length, "chronos_base")
results_dict5 = chronos_zero_shot(input_lengths, prediction_length, "chronos_large")

2025-12-03 15:16:31,999 - Processing input length: 24
No path specified. Models will be saved in: "AutogluonModels/ag-20251203_151632"
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20251203_151632'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #14~24.04.1-Ubuntu SMP Fri Oct  3 20:52:11 UTC 2025
CPU Count:          16
GPU Count:          1
Memory Avail:       111.65 GB / 125.71 GB (88.8%)
Disk Space Avail:   197.89 GB / 1003.85 GB (19.7%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_mod

In [42]:
# after COVID-19
results_dict1_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_tiny")
results_dict2_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")
results_dict3_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_small")
results_dict4_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_base")
results_dict5_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_large")

2025-12-04 10:31:11,862 - Processing input length: 24
No path specified. Models will be saved in: "AutogluonModels/ag-20251204_103111"
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20251204_103111'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #14~24.04.1-Ubuntu SMP Fri Oct  3 20:52:11 UTC 2025
CPU Count:          16
GPU Count:          1
Memory Avail:       110.45 GB / 125.71 GB (87.9%)
Disk Space Avail:   196.45 GB / 1003.85 GB (19.6%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_mod

In [43]:
def cal_metrics(df_true, df_pred, input_len):
    df_merged = pd.merge(df_true, df_pred, on=['item_id', 'timestamp'], how='inner')

    # Step 3: Assign prediction step numbers (1 to 6) within each 'item_id'
    df_merged['step'] = df_merged.groupby('item_id').cumcount() + 1

    # Step 4: Calculate error metrics
    df_merged['error'] = df_merged['mean'] - df_merged['target']
    df_merged['abs_error'] = df_merged['error'].abs()
    df_merged['squared_error'] = df_merged['error'] ** 2

    # Handle division by zero in MAPE calculation by adding a very small number (epsilon)
    epsilon = 1e-10
    df_merged['abs_percentage_error'] = df_merged['abs_error'] / (df_merged['target'].abs() + epsilon)

    # Step 5: Calculate overall metrics
    overall_mse = df_merged['squared_error'].mean()
    overall_rmse = np.sqrt(overall_mse)
    overall_mae = df_merged['abs_error'].mean()
    overall_mape = df_merged['abs_percentage_error'].mean() * 100

    print("Overall Evaluation Metrics:")
    print(f"MSE: {overall_mse:.4f}")
    print(f"RMSE: {overall_rmse:.4f}")
    print(f"MAE: {overall_mae:.4f}")
    print(f"MAPE: {overall_mape:.2f}%")

    # Step 6: Calculate metrics for each prediction step
    metrics_by_step = df_merged.groupby('step').agg({
      'squared_error': 'mean',
      'abs_error': 'mean',
      'abs_percentage_error': 'mean'
    }).reset_index()

    metrics_by_step['MSE'] = metrics_by_step['squared_error']
    metrics_by_step['RMSE'] = np.sqrt(metrics_by_step['squared_error'])
    metrics_by_step['MAE'] = metrics_by_step['abs_error']
    metrics_by_step['MAPE'] = metrics_by_step['abs_percentage_error'] * 100

    # Select relevant columns and round for better readability
    metrics_by_step = metrics_by_step[['step', 'MSE', 'RMSE', 'MAE', 'MAPE']]
    metrics_by_step.insert(0, 'input_len', input_len)

    return metrics_by_step

In [44]:
def metrics_results(results_dict):
    all_dfs = []
    for length in input_lengths:
        #print(f"Processing input length: {length}")
    
        predict = results_dict[length]['Predicted_Results']
        truth = results_dict[length]['Gound_Truth']
        evaluation_df = cal_metrics(predict, truth, length)
        #display(evaluation_df)
        all_dfs.append(evaluation_df)
    final_df = pd.concat(all_dfs, ignore_index=True)
    return final_df

In [32]:
# entire data metrics
all_metrics1 = metrics_results(results_dict1)
all_metrics2 = metrics_results(results_dict2)
all_metrics3 = metrics_results(results_dict3)
all_metrics4 = metrics_results(results_dict4)
all_metrics5 = metrics_results(results_dict5)

Overall Evaluation Metrics:
MSE: 2244.3584
RMSE: 47.3747
MAE: 32.5315
MAPE: 34.92%
Overall Evaluation Metrics:
MSE: 1983.0282
RMSE: 44.5312
MAE: 29.4247
MAPE: 31.65%
Overall Evaluation Metrics:
MSE: 1942.6707
RMSE: 44.0757
MAE: 28.8227
MAPE: 30.52%
Overall Evaluation Metrics:
MSE: 1863.6288
RMSE: 43.1698
MAE: 28.0905
MAPE: 29.84%
Overall Evaluation Metrics:
MSE: 1792.4049
RMSE: 42.3368
MAE: 27.4833
MAPE: 29.35%
Overall Evaluation Metrics:
MSE: 1777.3215
RMSE: 42.1583
MAE: 27.1535
MAPE: 28.47%
Overall Evaluation Metrics:
MSE: 1758.6743
RMSE: 41.9366
MAE: 26.8179
MAPE: 28.10%
Overall Evaluation Metrics:
MSE: 1741.5044
RMSE: 41.7313
MAE: 26.2983
MAPE: 27.43%
Overall Evaluation Metrics:
MSE: 1704.5380
RMSE: 41.2861
MAE: 25.8895
MAPE: 27.12%
Overall Evaluation Metrics:
MSE: 1728.1266
RMSE: 41.5707
MAE: 25.8605
MAPE: 27.02%
Overall Evaluation Metrics:
MSE: 1729.2038
RMSE: 41.5837
MAE: 25.9263
MAPE: 27.25%
Overall Evaluation Metrics:
MSE: 1740.1705
RMSE: 41.7154
MAE: 25.9455
MAPE: 27.24%
Over

In [34]:
# entire data results
dfs = [all_metrics1, all_metrics2, all_metrics3, all_metrics4, all_metrics5]
names = ["tiny", "mini", "small", "base", "large"]
stacked_df = pd.concat(
    [   # Add the identifier column first
        pd.concat(
            [pd.DataFrame({"model": [name] * len(df)}), df.reset_index(drop=True)],
            axis=1)
        for df, name in zip(dfs, names)
    ],ignore_index=True)
stacked_df
# stacked_df.to_csv('all_metrics_chronos_entire_data.csv', index=False)  

In [45]:
# after COVID-19 metrics
all_metrics1_short = metrics_results(results_dict1_short)
all_metrics2_short = metrics_results(results_dict2_short)
all_metrics3_short = metrics_results(results_dict3_short)
all_metrics4_short = metrics_results(results_dict4_short)
all_metrics5_short = metrics_results(results_dict5_short)

Overall Evaluation Metrics:
MSE: 1168.9452
RMSE: 34.1898
MAE: 26.1718
MAPE: 23.07%
Overall Evaluation Metrics:
MSE: 909.5197
RMSE: 30.1582
MAE: 22.7534
MAPE: 19.67%
Overall Evaluation Metrics:
MSE: 861.9512
RMSE: 29.3590
MAE: 21.8325
MAPE: 18.43%
Overall Evaluation Metrics:
MSE: 765.9914
RMSE: 27.6765
MAE: 20.9540
MAPE: 17.87%
Overall Evaluation Metrics:
MSE: 712.0625
RMSE: 26.6845
MAE: 19.9542
MAPE: 17.02%
Overall Evaluation Metrics:
MSE: 685.7339
RMSE: 26.1865
MAE: 19.5028
MAPE: 16.39%
Overall Evaluation Metrics:
MSE: 668.2225
RMSE: 25.8500
MAE: 19.1211
MAPE: 15.92%
Overall Evaluation Metrics:
MSE: 573.1729
RMSE: 23.9410
MAE: 17.5627
MAPE: 14.59%
Overall Evaluation Metrics:
MSE: 503.7713
RMSE: 22.4449
MAE: 16.3266
MAPE: 13.32%
Overall Evaluation Metrics:
MSE: 414.4279
RMSE: 20.3575
MAE: 15.2574
MAPE: 12.43%
Overall Evaluation Metrics:
MSE: 402.5810
RMSE: 20.0644
MAE: 15.1165
MAPE: 11.93%
Overall Evaluation Metrics:
MSE: 367.0771
RMSE: 19.1593
MAE: 14.6875
MAPE: 11.65%
Overall Evaluat

In [47]:
# after COVID-19 results
dfs = [all_metrics1_short, all_metrics2_short, all_metrics3_short, all_metrics4_short, all_metrics5_short]
names = ["tiny", "mini", "small", "base", "large"]
stacked_df = pd.concat(
    [   # Add the identifier column first
        pd.concat(
            [pd.DataFrame({"model": [name] * len(df)}), df.reset_index(drop=True)],
            axis=1)
        for df, name in zip(dfs, names)
    ],ignore_index=True)
stacked_df
# stacked_df.to_csv('all_metrics_chronos_after_covid.csv', index=False)  